# Visitation Number Mock Data

**Hourly room entries**: 0 to 28 per patient (median, 5.5)\
\
**visitors**: 0 to 18 per hour (median, 3.5)\
\
Nurses (45%), visitors (23%), medical staff (17%), nonclinical staff (7%), and other clinical staff (4%)\
\
**Visits** lasted 1 to 124 minutes (median, 3 minutes for all groups)

In [1]:
from random import randint

# Mock Visitation Numbers for One Visitor
def mockVisitationData(period, startDay = 0): # period is in days,startDay: 0 represents monday, 6 represents Sunday
    numberOfVisitors = [randint(1,6) for i in range(period)] # starts from day 1, a Monday
    
    # account for spikes in weekends
    for i in range(period):
        if (i - startDay) % 7 >= 5:
            numberOfVisitors[i] += randint(3,6)
    
    #account for spikes in public holidays
    publicHolidays = [0, 24, 25, 99, 120, 126, 142, 143, 211, 221, 317, 358]
    for i in range(period):
        if i in publicHolidays:
            numberOfVisitors[i] += randint(3,10)
            
    return numberOfVisitors

data = mockVisitationData(7)
print(data)

[15, 3, 2, 4, 5, 10, 9]


In [14]:
import numpy as np
import sklearn
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from math import floor, sqrt

from keras.models import Model
from keras.layers import Dense, Input

from statsmodels.tsa.arima_model import ARIMA

def FormatDataForTraining(HistoryData, window):
    HistoryData = [0] + HistoryData
    DifferencedData = [HistoryData[i+1] - HistoryData[i] for i in range(len(HistoryData)-1)]
    train_x = []
    for i in range(window, len(DifferencedData)):
        train_x.append(DifferencedData[i - window : i])
    
    train_y = [i for i in DifferencedData[window:]]
    
    
    print("\n==Data==")
    print("First set of data:", HistoryData[:window+1])
    print("train_x:", train_x[0])
    print("train_y:", train_y[0])
    print("==End Data==\n")
    
    return train_x, train_y

def train_neural_network(train_x, train_y, window):
    visible = Input((window))
    hidden = Dense(64, activation="relu")(visible)
    hidden = Dense(32, activation="relu")(hidden)
    hidden = Dense(1)(hidden)
    model = Model(visible, hidden)
    print("\n==neural network==")
    model.compile(optimizer='adam', loss='MeanSquaredError', metrics=['accuracy'])
    model.fit(train_x ,train_y, epochs=25)
    print(model.summary())
    print('==end neural network==\n')
    return model

def train_ARIMA(HistoryData):
    model = ARIMA(HistoryData, order=(28,1,0))
    model_fit = model.fit(disp=0)
    print(model_fit.summary())
    return model_fit

def predict(model, test_x):
    predictions = model.predict(test_x)
    return predictions

def predictNextValue(model, array):
    prediction = model.predict(array)
    return prediction
    
def myModelTestRMSE(model, test_data):
    test_x, test_y = FormatDataForTraining(test_data, 28)
    predictions = predict(model, test_x)
    assert len(predictions) == len(test_y)
    SE = [(test_y[i] - predictions[i]) ** 2 for i in range(len(predictions))]
    RMSE = (sum(SE) / len(predictions)) ** 0.5
    return RMSE
    
def RunTimeSeriesModel(HistoryData, window=28): # THE WINDOW MUST BE SMALLER THAN LEN OF HISTORY DATA)
    # History Data contains visitation data usually for the past year
    print("----Running Time Series Model----\n\n")
    # create training set and testing set
    train_x, train_y = FormatDataForTraining(HistoryData, window)
    model = train_neural_network(train_x, train_y, window)
#     model = train_ARIMA(HistoryData)
    print("\n==Predictions==")
    test_data = mockVisitationData(365)
    RMSE = myModelTestRMSE(model, test_data)
    print("RMSE:", RMSE)
    print("==END Predictions==\n")
    
    print("\n== Single Array of size window Predictions==")
    singleArray = mockVisitationData(29)
    array_x, array_y = FormatDataForTraining(singleArray, 28)
    prediction = predictNextValue(model, array_x)
    print("Actual Value:", singleArray[-1])
    SUMMATION = sum(array_x[0]) + prediction[0][0]
    print("Prediction Value:", SUMMATION)
    print("==END Predictions==\n")
    print("\n\n----END Time Series Model----")
    
singleVisitorData = mockVisitationData(10000)
RunTimeSeriesModel(singleVisitorData, 28)

----Running Time Series Model----



==Data==
First set of data: [0, 12, 5, 1, 3, 2, 10, 9, 3, 3, 2, 6, 2, 7, 7, 4, 6, 1, 3, 5, 10, 11, 3, 1, 2, 12, 8, 12, 4]
train_x: [12, -7, -4, 2, -1, 8, -1, -6, 0, -1, 4, -4, 5, 0, -3, 2, -5, 2, 2, 5, 1, -8, -2, 1, 10, -4, 4, -8]
train_y: 0
==End Data==


==neural network==
Epoch 1/25
312/312 [==============================] - 0s 662us/step - loss: 6.3417 - accuracy: 0.1283
Epoch 2/25
312/312 [==============================] - 0s 663us/step - loss: 4.5957 - accuracy: 0.1354
Epoch 3/25
312/312 [==============================] - 0s 667us/step - loss: 4.1675 - accuracy: 0.1334
Epoch 4/25
312/312 [==============================] - 0s 677us/step - loss: 3.9431 - accuracy: 0.1354
Epoch 5/25
312/312 [==============================] - 0s 667us/step - loss: 3.7925 - accuracy: 0.1376
Epoch 6/25
312/312 [==============================] - 0s 672us/step - loss: 3.6733 - accuracy: 0.1362
Epoch 7/25
312/312 [==============================] - 0s 670us/step - loss: